In [7]:
import os
os.chdir('c:/SAVI MSI/Complete Data Science/kidney-disease-classification-deep-learning-project')
%pwd

'c:\\SAVI MSI\\Complete Data Science\\kidney-disease-classification-deep-learning-project'

In [8]:
os.environ['MLFLOW_TRACKING_URI']="https://dagshub.com/sakshamwalia/kidney-disease-classification-deep-learning-project.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']="sakshamwalia"
os.environ['MLFLOW_TRACKING_PASSWORD']="549c5cc4ae00fd52e55b92e68cec5dc45a19bc65"

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model:Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [10]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH) :
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_evaluation_config(self) -> EvaluationConfig:
        training = self.config.training
        
        eval_config = EvaluationConfig(
            path_of_model='artifacts/training/model.h5',
            training_data='artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone',
            mlflow_uri='https://dagshub.com/sakshamwalia/kidney-disease-classification-deep-learning-project.mlflow',
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from keras.preprocessing.image import ImageDataGenerator

class Evaluation:
    def __init__(self, config: EvaluationConfig) :
        self.config = config
        
    def _valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1.0/255,
            validation_split = 0.30
        )
        
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
        )
        
        valid_datagenerator = ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            **dataflow_kwargs
        )
    
    # static method - this function is not a class function as it is not containing 'self', basically this function is an independent function and we can access it from anywhere
    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()
        
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != 'file':
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name='VGG16Model')
                
            else:
                mlflow.keras.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(config=eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-02-21 02:44:20,992 : INFO : common : yaml file : config\config.yaml loaded successfully]
[2024-02-21 02:44:20,995 : INFO : common : yaml file : params.yaml loaded successfully]
[2024-02-21 02:44:20,997 : INFO : common : created directory at : artifacts]
Found 2207 images belonging to 2 classes.
69/69 [==============================] - 114s 2s/step - loss: 15.6229 - Accuracy: 0.3099
[2024-02-21 02:46:15,502 : INFO : common : json file saved at : scores.json]


2024/02/21 02:46:17 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-02-21 02:46:18,572 : WARNING : save : Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
[2024-02-21 02:46:19,249 : INFO : builder_impl : Assets written to: C:\Users\Savi\AppData\Local\Temp\tmpfvcejwyh\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/02/21 02:47:11 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 5
Created version '5' of model 'VGG16Model'.
